# This notebook discribes the problems during plog reaction validation improvement

As we suggested on [enhancement repo #42](https://github.com/Cantera/enhancements/issues/42#issuecomment-641341008), I am trying to change the Cantera PLOG reaction validation function to raise all the invalid reactions and their invalid conditions from the source code to replace the python function we built before. The revised code is posted on the [forked repo](https://github.com/12Chao/cantera/blob/d8a5df25dc4849292576e73b085f2224817e9848/src/kinetics/RxnRates.cpp#L91). Below is the discription of the problems.
    

There are 2 problematic PLOG reactions in the input [yaml file](https://github.com/12Chao/myproject/blob/master/plog_validation_error/PLOG.yaml) as shown below, the first reation has negative sum of reaction rate coefficients at 1 atm, 500K and 1000k; 3.16 atm, 200K, 500K, 1000K, 2000K, 5000K, 10000K(all the temperature points). The second reaction is invalid at 100 atm, 500K. 

My expectation is to raise a `CanteraError` to throw an error including all these invalid reations and their conditions. From my initial thought, I was trying to create a string vector from where the iteration of all the PLOG reactions starts, and change the `Plog::validate` to add the error message to that vector if a validation failed instead throw the `CanteraError` right away. Then throw all the strings in the vector if the vector is not empty when the iteration ends. However, I did not find that kind of iteration of all the PLOG reactions in the source code. The `Plog::validate` is defined in `RxnRates.cpp` line 91 and called in `Reaction.cpp` line 737, but it seems that there is no obvious 'iteration' like a `for loop`, so **my question is how all the `Plog reactions` get validated? If it is feasible to handle the error like I metioned above?**


- equation: CH2CHOO <=> CO + CH3O  # Reaction 566
  type: pressure-dependent-Arrhenius
  rate-constants:
  - {P: 0.01 atm, A: 5.2e+33, b: -7.92, Ea: 3.132e+04}
  - {P: 0.01 atm, A: 2.31e+129, b: -41.86, Ea: 4.585e+04}
  - {P: 0.1 atm, A: 1.26e+98, b: -27.09, Ea: 6.406e+04}
  - {P: 0.1 atm, A: 2.42e+28, b: -5.99, Ea: 3.054e+04}
  - {P: 0.316 atm, A: 1.8e+33, b: -7.27, Ea: 3.376e+04}
  - {P: 0.316 atm, A: 8.69e-50, b: 16.63, Ea: -3900.0}
  - <span style="color: red">{P: 1.0 atm, A: -3.83e+33, b: -7.2, Ea: 3.51e+04}</span>
  - <span style="color: red">{P: 1.0 atm, A: 1.19e-39, b: 13.61, Ea: -1317.0}</span>
  - <span style="color: red">{P: 3.16 atm, A: -1.28e+79, b: -19.61, Ea: 7.487e+04}</span>
  - <span style="color: red">{P: 3.16 atm, A: -8.8e+86, b: -23.08, Ea: 6.101e+04}</span>
  - {P: 10.0 atm, A: 4.07e+32, b: -6.62, Ea: 3.721e+04}
  - {P: 10.0 atm, A: 1270.0, b: 1.44, Ea: 1.866e+04}
  - {P: 31.6 atm, A: 6.86e+44, b: -10.04, Ea: 4.703e+04}
  - {P: 31.6 atm, A: 1.97e+17, b: -2.23, Ea: 2.859e+04}
  - {P: 100.0 atm, A: -1.07e+04, b: 1.33, Ea: 1.562e+04}
  - {P: 100.0 atm, A: 1.16e-07, b: 4.81, Ea: 1.201e+04}
  note: 'Goldsmith et al., 2015, J. Phys. Chem. A 2015, 119, 7766-7779 (DOI:
    10.1021/acs.jpca.5b01088)'
- equation: CH2CHOO <=> CO2 + CH3  # Reaction 567
  type: pressure-dependent-Arrhenius
  rate-constants:
  - {P: 0.01 atm, A: 5.09e+33, b: -7.95, Ea: 3.129e+04}
  - {P: 0.01 atm, A: 4.2e+122, b: -39.75, Ea: 4.364e+04}
  - {P: 0.1 atm, A: 1.21e+118, b: -33.13, Ea: 7.379e+04}
  - {P: 0.1 atm, A: 1.96e+29, b: -6.29, Ea: 3.092e+04}
  - {P: 0.316 atm, A: 8.56e+32, b: -7.21, Ea: 3.355e+04}
  - {P: 0.316 atm, A: 5.1e-66, b: 21.37, Ea: -1.111e+04}
  - {P: 1.0 atm, A: 3.27e+33, b: -7.22, Ea: 3.499e+04}
  - {P: 1.0 atm, A: 1.76e-47, b: 15.85, Ea: -5283.0}
  - {P: 3.16 atm, A: 3.49e-79, b: 25.01, Ea: -2.102e+04}
  - {P: 3.16 atm, A: 3.82e+32, b: -6.8, Ea: 3.569e+04}
  - {P: 10.0 atm, A: 8.16e+32, b: -6.76, Ea: 3.727e+04}
  - {P: 10.0 atm, A: 4.62, b: 2.1, Ea: 1.717e+04}
  - {P: 31.6 atm, A: 7.01e+37, b: -8.06, Ea: 4.22e+04}
  - {P: 31.6 atm, A: 3.49e+14, b: -1.58, Ea: 2.647e+04}
  - <span style="color: red">{P: 100.0 atm, A: -2510.0, b: 1.41, Ea: 1.442e+04}</span>
  - <span style="color: red">{P: 100.0 atm, A: 4.05e-09, b: 5.14, Ea: 1.048e+04}</span>


## To verify the negative reaction rate coefficients

In [1]:
import math
R = 1.98720425864083
def k(arrhenius_parameters, T):
    rate_constant = arrhenius_parameters['A'] * (T ** arrhenius_parameters['b']) * math.exp(-arrhenius_parameters['Ea']/T/R)
    return(rate_constant)
def throw_invalid_reactions(arr1, arr2, temperature_range, equation):
    for t in temperature_range:
        k1 = arr1['A'] * (t ** arr1['b']) * math.exp(-arr1['Ea']/t/R)
        k2 = arr2['A'] * (t ** arr2['b']) * math.exp(-arr2['Ea']/t/R)
        if k1+k2 < 0:
            print('{} has invalid reaction rate coefficient {} at \nP = {} atm,T = {}'.format(equation, k1+k2, arr1['P'], t))

In [2]:
equation1 = 'CH2CHOO <=> CO + CH3O'
arrhenius_1 = {'P': 1.0, 'A': -3.83e+33, 'b': -7.2, 'Ea': 3.51e+04}
arrhenius_2 = {'P': 1.0, 'A': 1.19e-39, 'b': 13.61, 'Ea': -1317.0}
arr_3 = {'P': 3.16, 'A': -1.28e+79, 'b': -19.61, 'Ea': 7.487e+04}
arr_4 = {'P': 3.16, 'A': -8.8e+86, 'b': -23.08, 'Ea': 6.101e+04}

equation2 = 'CH2CHOO <=> CO2 + CH3'
arr_5 = {'P': 100.0, 'A': -2510.0, 'b': 1.41, 'Ea': 1.442e+04}
arr_6 = {'P': 100.0, 'A': 4.05e-09, 'b': 5.14, 'Ea': 1.048e+04}

T = [200, 500, 1000, 2000, 5000, 10000]
throw_invalid_reactions(arrhenius_1, arrhenius_2, T, equation1)
print('================================')
throw_invalid_reactions(arr_3, arr_4, T, equation1)
print('=================================')
throw_invalid_reactions(arr_5, arr_6, T, equation2)

CH2CHOO <=> CO + CH3O has invalid reaction rate coefficient -0.04015566549862874 at 
P = 1.0 atm,T = 500
CH2CHOO <=> CO + CH3O has invalid reaction rate coefficient -20367.48086706905 at 
P = 1.0 atm,T = 1000
CH2CHOO <=> CO + CH3O has invalid reaction rate coefficient -1.4771200414049596e-33 at 
P = 3.16 atm,T = 200
CH2CHOO <=> CO + CH3O has invalid reaction rate coefficient -0.009668296450743921 at 
P = 3.16 atm,T = 500
CH2CHOO <=> CO + CH3O has invalid reaction rate coefficient -31714.568070527945 at 
P = 3.16 atm,T = 1000
CH2CHOO <=> CO + CH3O has invalid reaction rate coefficient -1571000.9902456335 at 
P = 3.16 atm,T = 2000
CH2CHOO <=> CO + CH3O has invalid reaction rate coefficient -1985.7563712502777 at 
P = 3.16 atm,T = 5000
CH2CHOO <=> CO + CH3O has invalid reaction rate coefficient -0.10738926054236393 at 
P = 3.16 atm,T = 10000
CH2CHOO <=> CO2 + CH3 has invalid reaction rate coefficient -0.053561302341967654 at 
P = 100.0 atm,T = 500


To compromise, `warn_user` is temporarily used to throw a warning instead a `CanteraError` so that all the problematic reactions get raised. But another problem is that I have some redundant error messages involving the reaction conditions with valid reaction rate coefficients. The error messages are shown as below:

## This is the error message before the change

In [3]:
import sys
sys.executable = '/home/chao/anaconda3/envs/cantera38/lib/python3.8/site-packages'
python_path = ['', '/home/chao/RMG-Py', '/home/chao', '/home/chao/anaconda3/envs/cantera38/lib/python38.zip', '/home/chao/anaconda3/envs/cantera38/lib/python3.8', '/home/chao/anaconda3/envs/cantera38/lib/python3.8/lib-dynload', '/home/chao/anaconda3/envs/cantera38/lib/python3.8/site-packages']
for path in python_path:
    sys.path.append(path)
import cantera as ct

In [4]:
ct.Solution('/home/chao/cc_test/PLOG.yaml')

For species HOCHO, discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  -40.36035916666667
	Value computed using high-temperature polynomial: -38.16786118416666



CanteraError: 
***********************************************************************
CanteraError thrown by Plog::validate:
Invalid rate coefficient for reaction 'CH2CHOO <=> CH3O + CO'
at P = 101324.99999999999, T = 500.0
***********************************************************************


## This is the error message after change

In [1]:
import sys
sys.executable = '/home/chao/anaconda3/envs/cantera38/lib/python3.8/site-packages'
python_path = ['/home/chao/cantera/build/python', '/home/chao/RMG-Py', '/home/chao', '/home/chao/anaconda3/envs/cantera38/lib/python38.zip', '/home/chao/anaconda3/envs/cantera38/lib/python3.8', '/home/chao/anaconda3/envs/cantera38/lib/python3.8/lib-dynload', '/home/chao/anaconda3/envs/cantera38/lib/python3.8/site-packages']
for path in python_path:
    sys.path.append(path)
import cantera as ct

In [2]:
ct.Solution('/home/chao/cc_test/PLOG.yaml')

For species HOCHO, discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  -40.36035916666667
	Value computed using high-temperature polynomial: -38.16786118416666


Invalid rate coefficient for reaction 'CH2CHOO <=> CH3O + CO'
at P = 101324.99999999999 , T = 500.0
Invalid rate coefficient for reaction 'CH2CHOO <=> CH3O + CO'
at P = 101324.99999999999 , T = 1000.0
Invalid rate coefficient for reaction 'CH2CHOO <=> CH3O + CO'
at P = 320187.0000000002 , T = 200.0
Invalid rate coefficient for reaction 'CH2CHOO <=> CH3O + CO'
at P = 320187.0000000002 , T = 500.0
Invalid rate coefficient for reaction 'CH2CHOO <=> CH3O + CO'
at P = 320187.0000000002 , T = 1000.0
Invalid rate coefficient for reaction 'CH2CHOO <=> CH3O + CO'
at P = 320187.0000000002 , T = 2000.0
Invalid rate coefficient for reaction 'CH2CHOO <=> CH3O + CO'
at P = 320187.0000000002 , T = 5000.0
Invalid rate coefficient for reaction 'CH2CHOO <=> CH3O + CO'
at P = 320187.0000000002 , T =

As shown in the warning,
```
Invalid rate coefficient for reaction 'CH2CHOO <=> CH3O + CO'
at P = 1013249.9999999992 , T = 200.0
Invalid rate coefficient for reaction 'CH2CHOO <=> CH3O + CO'
at P = 1013249.9999999992 , T = 500.0
Invalid rate coefficient for reaction 'CH2CHOO <=> CH3O + CO'
at P = 1013249.9999999992 , T = 1000.0
Invalid rate coefficient for reaction 'CH2CHOO <=> CH3O + CO'
at P = 1013249.9999999992 , T = 2000.0
Invalid rate coefficient for reaction 'CH2CHOO <=> CH3O + CO'
at P = 1013249.9999999992 , T = 5000.0
Invalid rate coefficient for reaction 'CH2CHOO <=> CH3O + CO'
at P = 1013249.9999999992 , T = 10000.0
```
and 
```
Invalid rate coefficient for reaction 'CH2CHOO <=> CH3 + CO2'
at P = 0.0 , T = 500.0
```
are not expected, but I cannot figure out how to fix the bug. It would be appreciated if anyone could give me a hint about how to handle the error and how to fix the bug. Please forgive me if there is any stupid error in the code.

